In [1]:
### Extracting the data

In [4]:
import networkx as nx
import numpy as np

In [5]:
cities_dist = np.loadtxt("sgb128_dist.txt", dtype = int)
cities_xy = np.loadtxt("sgb128_xy.txt")
print(cities_dist)
print()
print(cities_xy)

OSError: sgb128_dist.txt not found.

In [3]:
print(f"cities_dist Shape:",cities_dist.shape)
print(f"cities_xy Shape:",cities_xy.shape)

cities_dist Shape: (128, 128)
cities_xy Shape: (128, 2)


### Loading the dataset in a NetworkX graph

In [4]:
# Adding new nodes and their X,Y cordinates to the graph
G = nx.Graph()
G.add_nodes_from(list(range(0, 128)))
nx.set_node_attributes(G, cities_xy, "xy")
xy = nx.get_node_attributes(G,'xy')[0]

In [5]:
# Helper function to determine the straight line distance between the two nodes
def straight_line_distance(a, b):
    return (((xy[a][0] - xy[b][0]) ** 2 + (xy[a][1] - xy[b][1]) ** 2) ** 0.5)

In [6]:
# Adding nodes only if distance > 230
for x in range(0, 128):
    for y in range(0, 128):
        if straight_line_distance(x,y) < 230:
            G.add_weighted_edges_from([(x, y, cities_dist[y][x])])

In [7]:
# Final graph with attributes
nx.get_edge_attributes(G,'weight')

{(0, 0): 0,
 (0, 7): 1510,
 (0, 15): 1017,
 (0, 27): 326,
 (0, 37): 417,
 (0, 47): 435,
 (0, 49): 775,
 (0, 61): 675,
 (0, 91): 2493,
 (0, 127): 348,
 (1, 1): 0,
 (1, 22): 393,
 (1, 70): 1278,
 (1, 71): 589,
 (2, 2): 0,
 (2, 17): 1783,
 (2, 28): 2873,
 (2, 54): 2295,
 (2, 68): 1830,
 (2, 78): 2093,
 (2, 101): 722,
 (3, 3): 0,
 (3, 25): 958,
 (3, 66): 1803,
 (3, 81): 1187,
 (3, 106): 1412,
 (3, 111): 1384,
 (4, 4): 0,
 (4, 20): 708,
 (4, 21): 1360,
 (4, 24): 1178,
 (4, 60): 929,
 (4, 115): 2056,
 (4, 117): 1205,
 (4, 122): 260,
 (5, 5): 0,
 (5, 8): 2956,
 (5, 12): 1408,
 (5, 57): 715,
 (5, 63): 367,
 (5, 113): 1367,
 (5, 118): 1647,
 (5, 119): 120,
 (6, 6): 0,
 (6, 34): 1225,
 (7, 7): 0,
 (7, 9): 1305,
 (7, 11): 369,
 (7, 15): 203,
 (7, 26): 2936,
 (7, 27): 976,
 (7, 37): 1356,
 (7, 61): 944,
 (7, 63): 447,
 (7, 97): 3057,
 (7, 113): 837,
 (7, 118): 2725,
 (7, 119): 577,
 (7, 126): 345,
 (8, 8): 0,
 (8, 57): 1151,
 (8, 113): 1204,
 (9, 9): 0,
 (9, 11): 1203,
 (9, 25): 512,
 (9, 26): 105

### Helper functions for actual code

In [8]:
# Decorator function to measure the time required to excecute a code
import time
import functools

def timer(func):
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        tic = time.perf_counter()
        value = func(*args, **kwargs)
        toc = time.perf_counter()
        elapsed_time = toc - tic
        print(f"Elapsed time: {elapsed_time:0.4f} seconds")
        return value
    return wrapper_timer

In [9]:
# Heuristic function given as input to a* star code with 
def heuristic_func(a, b,weight=0):
    return straight_line_distance(a,b) * weight

In [10]:
# Copy of NetworkX library code with changes so that the algorithm accepts a Heuristic function with 3 inputs
from heapq import heappush, heappop
from itertools import count

import networkx as nx
from networkx.algorithms.shortest_paths.weighted import _weight_function

__all__ = ["astar_path_my", "astar_path_length_2"]


def astar_path_my(G, source, target, heuristic=None, weight="weight",ww=0):
    """Returns a list of nodes in a shortest path between source and target
    using the A* ("A-star") algorithm.
    """
    if source not in G or target not in G:
        msg = f"Either source {source} or target {target} is not in G"
        raise nx.NodeNotFound(msg)

    if heuristic is None:
        # The default heuristic is h=0 - same as Dijkstra's algorithm
        def heuristic(u, v,ww):
            return 0

    push = heappush
    pop = heappop
    weight = _weight_function(G, weight)
    c = count()
    queue = [(0, next(c), source, 0, None)]
    enqueued = {}
    explored = {}
    repeatedly_visited_nodes=0
    unique_set = set()
    
    while queue:
        # Pop the smallest item from queue.
        _, __, curnode, dist, parent = pop(queue)

        if curnode == target:
            path = [curnode]
            node = parent
            while node is not None:
                path.append(node)
                node = explored[node]
            path.reverse()
            print("Unique Nodes Explored Completely:", len(explored))
            print("Unique Nodes Visited:", len(unique_set))
            print("Nodes Visited (One node can be visited more than once):",repeatedly_visited_nodes)
            return path

        if curnode in explored:
            # Do not override the parent of starting node
            if explored[curnode] is None:
                continue

            # Skip bad paths that were enqueued before finding a better one
            qcost, h = enqueued[curnode]
            if qcost < dist:
                continue

        explored[curnode] = parent

        for neighbor, w in G[curnode].items():
            if neighbor not in explored:
                repeatedly_visited_nodes +=1
                unique_set.add(neighbor)
            ncost = dist + weight(curnode, neighbor, w)  
            if neighbor in enqueued:
                qcost, h = enqueued[neighbor]
                if qcost <= ncost:
                    continue
            else:
                h = heuristic(neighbor, target,ww)
            enqueued[neighbor] = ncost, h
            push(queue, (ncost + h, next(c), neighbor, ncost, curnode))

    raise nx.NetworkXNoPath(f"Node {target} not reachable from {source}")



def astar_path_length_2(G, source, target, heuristic=None, weight="weight",ww=0):
    """Returns the length of the shortest path between source and target using
    the A* ("A-star") algorithm.
    """
    if source not in G or target not in G:
        msg = f"Either source {source} or target {target} is not in G"
        raise nx.NodeNotFound(msg)

    weight = _weight_function(G, weight)
    path = astar_path_my(G, source, target, heuristic, weight,ww)
    return sum(weight(u, v, G[u][v]) for u, v in zip(path[:-1], path[1:])) , path


### Computing shortest path using Dijkstra

In [11]:
@timer
def get_dijkstra_results(G,start_index,stop_index):
    length, path = nx.single_source_dijkstra(G, 6, 16,weight='weight')
    print("Results for Dijkstra")
    print("---------------------------------------------------------")
    print("Path length:",length)
    print("Path Route:",path)

In [12]:
get_dijkstra_results(G,6,16)

Results for Dijkstra
---------------------------------------------------------
Path length: 11588
Path Route: [6, 34, 22, 71, 104, 77, 103, 51, 105, 49, 61, 118, 119, 113, 57, 56, 35, 52, 16]
Elapsed time: 0.0022 seconds


### Computing shortest path using A* and different heuristic weights

In [13]:
@timer
def get_astar_results(G,start_index,stop_index,ww):
    print("---------------------------------------------------------")
    length, path = astar_path_length_2(G, 6, 16, heuristic = heuristic_func,ww=ww)    
    print("Path length:",length)
    print("Path Route:",path)

In [14]:
heuristic_weights = [0, 0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for i in heuristic_weights:
    print(f"\nResults for Astart with weight {i}")
    get_astar_results(G,6,16,ww=i)


Results for Astart with weight 0
---------------------------------------------------------
Unique Nodes Explored Completely: 90
Unique Nodes Visited: 92
Nodes Visited (One node can be visited more than once): 314
Path length: 11588
Path Route: [6, 34, 22, 71, 104, 77, 103, 51, 105, 49, 61, 118, 119, 113, 57, 56, 35, 52, 16]
Elapsed time: 0.0019 seconds

Results for Astart with weight 0.5
---------------------------------------------------------
Unique Nodes Explored Completely: 90
Unique Nodes Visited: 92
Nodes Visited (One node can be visited more than once): 314
Path length: 11588
Path Route: [6, 34, 22, 71, 104, 77, 103, 51, 105, 49, 61, 118, 119, 113, 57, 56, 35, 52, 16]
Elapsed time: 0.0027 seconds

Results for Astart with weight 1
---------------------------------------------------------
Unique Nodes Explored Completely: 90
Unique Nodes Visited: 92
Nodes Visited (One node can be visited more than once): 314
Path length: 11588
Path Route: [6, 34, 22, 71, 104, 77, 103, 51, 105, 49

### Results

1. Shortest path length using Dijkstra is 11588. Shorted path using A* is also 11588 for weigths 0, 0.5, 1, 2, 3.
2. No of nodes visited decreases with the heuristic weight's associated. With a weight of 3 we are able to replicate the path length and route results for Disjkstra and still visit lesser number of nodes.
3. When compared based on excecution time, (note that the accuracy of python timer function in milliseconds is low) the time required to excecute the code generally reduces with increase in weights. So, keeping accuracy and excecution time in mind, we get optimum results for weigth = 3 with A* algorithm.